# ResNet

## Setup

Choose the target board. You may need to install the proper board files for the chosen board.

In [ ]:
## ZCU106
#board_name='zcu106'
#fpga_part='xczu7ev-ffvc1156-2-e'
 
## Ultra96
#board_name='ultra96'
#fpga_part='xczu3eg-sbva484-1-e'

## Pynq-Z1
board_name='pynqz1'
fpga_part='xc7z020clg400-1'

## Pynq-Z2
#board_name='pynqz2'
#fpga_part='xc7z020clg400-1'

## MiniZed
#board_name='minized'
#fpga_part='xc7z007sclg225-1'

##Cmod A7-35t
#board_name='cmoda735t'
#fpga_part='xc7a35tcpg236-1'

## Arty A7-100t
#board_name='artya7100t'
#fpga_part='xc7a100t-csg324-1'

## Arty A7-35t
#board_name='artya735t'
#fpga_part='xc7a35ticsg324-1L'

Add the project name. The notebook will create sub-directories for the Vivado projects with different models and configurations.

In [ ]:
acc_name='resnet'

Let's import the libraries, call the magic functions, and setup the environment variables.

In [ ]:
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects

import numpy as np

import hls4ml

from callbacks import all_callbacks
import plotting

import matplotlib.pyplot as plt
%matplotlib inline

import os
os.environ['PATH'] = '/tools/Xilinx/Vivado/2019.1/bin:' + os.environ['PATH']

def is_tool(name):
    from distutils.spawn import find_executable
    return find_executable(name) is not None

print('-----------------------------------')
if not is_tool('vivado_hls'):
    print('Xilinx Vivado HLS is NOT in the PATH')
else:
    print('Xilinx Vivado HLS is in the PATH')
print('-----------------------------------')

## Load the dataset

In [ ]:
#load processed test data
from sklearn.utils import shuffle
X_test = np.load('./test_data/resnet/X_test.npy', allow_pickle=True)
y_test = np.load('./test_data/resnet/y_test.npy', allow_pickle=True) # ground truth

In [ ]:
print(X_test.shape)
print(y_test.shape)

## Train or Load Model

In [ ]:
import keras_model
train = False
#not os.path.exists('model/KERAS_check_best_model.h5')
if train:
    model.compile(loss="mean_squared_error", optimizer="adam")
        
    print("Shape of training data element is: {}".format(train_data[0].shape))
    history = model.fit(train_data,
                        train_data,
                        epochs=100,
                        batch_size=512,
                        shuffle=true,
                        validation_split=0.1,
                        verbose=1,
                        callbacks=callbacks)
    

else:
    #model_file = "model/resnet/model_best_20210424.h5"
    
    # RN03
    #model_file = 'model/resnet/rn03/myquant.h5'
    #model_file = 'model/resnet/rn03/myquant_nosoftmax.h5'
    
    # RN06
    model_file = 'model/resnet/rn06/rushil2_nosoftmax.h5'
    
    if not os.path.exists(model_file):
        print("{} model not found at path ".format(model_file))

    model = keras_model.load_model(model_file)

model.summary()    

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')

**Select a portion of the model to debug**

## Check model sparisty

In [ ]:
w = model.layers[1].weights[0].numpy()
h, b = np.histogram(w, bins=100)
plt.figure(figsize=(7,7))
plt.bar(b[:-1], h, width=b[1]-b[0])
plt.semilogy()
print('% of zeros = {}'.format(np.sum(w==0)/np.size(w)))

## Check accuracy

Do not expect a good accuracy because of the low amount of neurons. I could have done better than this, but as long as it fits both Pynq-Z1 and MiniZed, it is fine with us.

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
import plotting

y_keras = model.predict(X_test)

classes = ['airplanes', 'cars', 'birds', 'cats', 'deer', 'dogs', 'frogs', 'horses', 'ships', 'trucks']

print("Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))

plt.figure(figsize=(9,9))
_ = plotting.plotMultiClassRoc(y_test, y_keras, classes)

import plotting # Import local package plotting.py
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=np.argmax(y_test, axis=1), y_pred=np.argmax(y_keras, axis=1))
plt.figure(figsize=(9,9))
_ = plotting.plot_confusion_matrix(cm, classes)


## Make an hls4ml configuration

Notice we're using `Strategy: Resource` for every layer, and `ReuseFactor: 64`. The Programmable Logic (FPGA part) of the Pynq-Z1 SoC is not big compared to VU9P type of parts.

We also use some settings which are good for QKeras.

Notice the `fpga_part:'xc7z020clg400-1'`.

In [ ]:
import yaml

def yaml_load(config):
    with open(config) as stream:
        param = yaml.safe_load(stream)
    return param


#training_config = yaml_load('model/resnet/config_20210424.yml')

# RN03
#training_config = yaml_load('model/resnet/rn03/myquant.yml')

# RN06
training_config = yaml_load('model/resnet/rn06/rushil2.yml')


# print("-----------------------------------")
#plotting.print_dict(training_config)
# print("-----------------------------------")

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')
hls_config['Model'] = {}
hls_config['Model']['ReuseFactor'] = training_config['convert']['ReuseFactor']
hls_config['Model']['Strategy'] = training_config['convert']['Strategy']
hls_config['Model']['Precision'] = training_config['convert']['Precision']

## DEBUG DEBUG DEBUG
##
## arrays keys and override_kets have been previously defined
## 
#for name in keys:
#    hls_config['LayerName'][name]['Trace'] = bool(training_config['convert']['Trace'])
#    hls_config['LayerName'][name]['ReuseFactor'] = training_config['convert']['ReuseFactor']
#    hls_config['LayerName'][name]['Precision'] = training_config['convert']['Precision']
#    if 'activation' in name:
#        hls_config['LayerName'][name]['Precision'] = training_config['convert']['PrecisionActivation']
## custom configs
#for name in override_keys:
#    hls_config['LayerName'][name].update(training_config['convert']['Override'][name])
## DEBUG DEBUG DEBUG


# THE ORIGINAL CONFIGURATION
for name in hls_config['LayerName'].keys():
    hls_config['LayerName'][name]['Trace'] = bool(training_config['convert']['Trace'])
    hls_config['LayerName'][name]['ReuseFactor'] = training_config['convert']['ReuseFactor']
    hls_config['LayerName'][name]['Precision'] = training_config['convert']['Precision']
    if 'activation' in name:
        hls_config['LayerName'][name]['Precision'] = training_config['convert']['PrecisionActivation']
# custom configs
for name in training_config['convert']['Override'].keys():
    hls_config['LayerName'][name].update(training_config['convert']['Override'][name])

# Enable tracing for all of the layers
for layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][layer]['Trace'] = True
    
print("-----------------------------------")
plotting.print_dict(hls_config)
print("-----------------------------------")

## Convert and Compile

You can set some target specific configurations:

- Define the `interface`, which for our current setup should always be `m_axi`.
- Define the  width of the AXI bus. For the time being, use `16` that is each clock cycle you transfer a single input or output value (`ap_fixed<16,*>`).
- Define the implementation. For the time being, use `serial`.

In [ ]:
interface = 'm_axi' # 's_axilite', 'm_axi', 'io_stream'
axi_width = 8 # 16, 32, 64
implementation = 'serial' # 'serial', 'dataflow'

In [ ]:
#output_dir='hls/' + '256x16x8x256_' + '_' + interface + '_' + str(axi_width) + '_' + implementation + '_prj'
output_dir='hls/' + board_name + '_' + acc_name + '_' + interface + '_' + str(axi_width) + '_' + implementation + '_prj' 

backend_config = hls4ml.converters.create_backend_config(fpga_part=fpga_part)
backend_config['ProjectName'] = acc_name
backend_config['KerasModel'] = model
backend_config['HLSConfig'] = hls_config
backend_config['OutputDir'] = output_dir
backend_config['Backend'] = 'Pynq'
backend_config['Interface'] = interface
backend_config['IOType'] = 'io_stream'
backend_config['AxiWidth'] = str(axi_width)
backend_config['Implementation'] = implementation
backend_config['ClockPeriod'] = 10

#print("-----------------------------------")
#plotting.print_dict(backend_config)
#print("-----------------------------------")

hls_model = hls4ml.converters.keras_to_hls(backend_config)
# hls_model = hls4ml.converters.convert_from_keras_model(model,
#                                                                 hls_config=hls_config,
#                                                                 output_dir=output_dir,
#                                                                 fpga_part=fpga_part,
#                                                                 clock_period=10,
#                                                                 io_type='io_parallel',
#                                                                 project_name='anomaly_detector')

_ = hls_model.compile()

## Profiling

In [ ]:
%%time
%matplotlib inline

# Run tracing on the test set for the hls model (fixed-point precision) 
hls4ml_pred, hls4ml_trace = hls_model.trace(np.ascontiguousarray(X_test))

# Run tracing on the test set for the Keras model (floating-point precision)
keras_trace = hls4ml.model.profiling.get_ymodel_keras(model, X_test)

# Run prediction on the test set for the hls model (fixed-point precision)
y_hls = hls_model.predict(np.ascontiguousarray(X_test))

_ = hls4ml.model.profiling.numerical(model=model, hls_model=hls_model, X=X_test)

Let's just print the output of the first layer, for the first sample, for both the Keras and hls4ml models.

## Manual Edits

Edit the following files:
- resnet_test.cpp
- firmware/resnet_axi.h

```
cd hls/pynqz1_resnet_m_axi_8_serial_prj
cp ../../patches/resnet/resnet_test.cpp ./resnet_test.cpp
cp ../../patches/resnet/resnet_axi.h ./firmware/resnet_axi.h
```

If you want to use 8-bit integer inputs, edit the following files instead:
```
cd hls/pynqz1_resnet_m_axi_8_serial_prj
cp ../../patches/resnet_div256/resnet_test.cpp ./resnet_test.cpp
cp ../../patches/resnet_div256/resnet_axi.h ./firmware/resnet_axi.h
cp ../../patches/resnet_div256/resnet_axi.h ./firmware/resnet_axi.h
```

### Run Vivado HLS c-sim for Integrity Test

Edit `build_prj.tcl`

```
array set opt {
  reset      0
  csim       1
  synth      0
  cosim      0
  validation 0
  export     0
  vsynth     0
} 
```

Then 
```
$ vivado_hls -f build_prj.tcl
```

### Generate .so Library

```
$ rm firmware/resnet-*.so 
$ bash build_lib.sh
```

## Prediction and Comparison


In [ ]:
print('-----------------------------------')
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))
print("hls4ml Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))))
print('-----------------------------------')

# Enable logarithmic scale on TPR and FPR axes 
logscale_tpr = False # Y axis
logscale_fpr = False # X axis

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.plotMultiClassRoc(y_test, y_keras, classes, logscale_tpr=logscale_tpr, logscale_fpr=logscale_fpr)
plt.gca().set_prop_cycle(None) # reset the colors
_ = plotting.plotMultiClassRoc(y_test, y_hls, classes, logscale_tpr=logscale_tpr, logscale_fpr=logscale_fpr, linestyle='--')

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], ls='-'),
         Line2D([0], [0], ls='--')]
from matplotlib.legend import Legend
leg = Legend(ax, lines, labels=['keras', 'hls4ml'],
            loc='center right', frameon=False)
_ = ax.add_artist(leg)


## Synthesis

In [ ]:
hls_model.build(csim=False, synth=True, export=True, vsynth=False)

hls4ml.report.read_vivado_report(output_dir)

## Resource Reference

See the resources availables on different boards.

```
+-----------------+---------+-------+--------+-------+-----+                    
|                 |               Resource                 |
+-----------------+---------+-------+--------+-------+-----+
|      Board      | BRAM_18K| DSP48E|   FF   |  LUT  | URAM|
+-----------------+---------+-------+--------+-------+-----+
|   PYNQ-Z1/Z2    |      280|    220|  106400|  53200|    0|
+-----------------+---------+-------+--------+-------+-----+
|     MiniZed     |      100|     66|   28800|  14400|    0|
+-----------------+---------+-------+--------+-------+-----+
``` 

## Generate .dat Files (Step 3)

The .dat files are used
- during the following `csim` step
- to generate the header files for SDK

In [ ]:
f = open(output_dir + '/tb_data/tb_input_features.dat', 'w')
# Save the first N images
N=10
for i in range(N):
    for j in range(X_test.shape[1]):
        for k in range(X_test.shape[2]):
            for c in range(X_test.shape[3]):
                f.write('{} '.format(X_test[i][j][k][c]))
    f.write('\n')
f.close()

f = open(output_dir + '/tb_data/tb_output_predictions.dat', 'w')
for i in range(N):
    for j in range(y_test.shape[1]):
        f.write('{} '.format(y_test[i][j]))
    f.write('\n')
f.close()

## Run Vivado HLS csim

At this step we generate simulation traces out from the hls4ml-model.

Run the following cell to run Vivado HLS GUI:

In [ ]:
!cd $output_dir && vivado_hls -p $acc_name\_prj

**IMPORTANT**  Click the button to `Run C Simulation`.

This will generate simulation traces with fixed-point arythmetic.

## Integrate IP in a Vivado Project and Generate Bitstream (Step 5)

In [ ]:
!cd sys/$board_name && make clean sys-gui ACC=$acc_name INTERFACE=$interface

**TODO** Tell the user how to visualize the `Block Diagram` to get a better understanding of the IP integration with both Zynq and MicroBlaze PS.

## Configure Software in Vivado SDK and Run HW/SW on the Board (Step 6)

Create Vivado SDK project.

- `make sdk` to configure an application with register polling
- `make sdk-irq` to configure an application with interrupts (default)

In [ ]:
!source /tools/Xilinx/Vivado/2019.1/settings64.sh && cd sdk/$board_name && make clean sdk ACC=$acc_name SAMPLE_COUNT=10

In [ ]:
!xterm -e "sleep 1 && source /tools/Xilinx/Vivado/2019.1/settings64.sh && cd sdk/$board_name && make gui && sleep infinity"


You can open a serial console, for example
```
sudo minicom -D /dev/ttyUSB0
```
and see something like

![serial-console](doc/serial_console.png)